**This code is running in sequence**. Please run the code one block by one block so you can easily grasp the concept. For the detail, you can read the readme.md  
Try to **take a quick look at Loan-Data.csv**, so we know what data we will process and how to clean it.

## Importing the Packages

In [1]:
import numpy as np

Code di bawah ini hanya untuk ***how we see the value on screen*** dan tidak memengaruhi hasil perhitungan.

In [2]:
np.set_printoptions(suppress = True, linewidth = 100, precision =2)

# supress = stop NumPy to using scientific notation to express numbers
# linewidth = extend the number of chars we fit in a single line output to 100
# precision = display only two digits after the decimal point

## Importing the Data

In [3]:
raw_data = np.genfromtxt("Loan-Data.csv", delimiter = ";", 
                         skip_header = 1,   # First row contains table heading 
                         autostrip = True)  # Remove excess white space which can distort our columns
raw_data

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

## Checking for Incomplete Data

Kita bisa mengetahui **terdapat 8805 data yang NaN** pada raw_data dengan isnan

In [4]:
np.isnan(raw_data).sum()

88005

Seperti pada *4.11 Pre-processing Data with Numpy* kita akan **membuat 2 variabel sebagai filler data NaN**  
Yaitu **temporary_fill** (nilai maksimum dari suatu dataset)  
dan **temporary_mean** (rata-rata sementara sebelum NaN dicleansing)

In [6]:
temporary_fill = np.nanmax(raw_data) + 1
temporary_mean = np.nanmean(raw_data, axis = 0)

C:\Users\Asus\AppData\Local\Temp\ipykernel_7676\671849336.py:2: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data, axis = 0)


Akan muncul **RuntimeWarning** yang memberitahu bahwa *ada sesuatu di nan.mean*, yaitu "mean of empty slice"  
NumPy memberitahu "Ada kolom yang meannya NaN nih, bener ga datanya?"  
**Yang berarti ada suatu kolom yang isinya memang string semua**. Karena tidak mungkin dari banyak data pada kolom tersebut, semuanya data kosong

In [7]:
temporary_mean

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

Bisa dibuktikan, ada beberapa kolom yang meannya memang NaN. **Karenanya, kita akan splitting data**.  
Akan ada array yang khusus menampung kolom string dan ada yang khusus menampung angka  
Tapi sebelum itu, **kita harus membuat temporary_stats** yang berisi nilai minimum, mean, dan maksimum sementara sebelum datanya diisi dengan filler yang kita buat di *temporary_fill*

In [8]:
temporary_stats = np.array([np.nanmin(raw_data, axis = 0),
                            temporary_mean,
                            np.nanmax(raw_data, axis = 0)])

C:\Users\Asus\AppData\Local\Temp\ipykernel_7676\1976739372.py:1: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data, axis = 0),
C:\Users\Asus\AppData\Local\Temp\ipykernel_7676\1976739372.py:3: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data, axis = 0)])


Abaikan RuntimeWarning, karena kita run data yang memang terdapat string  
**Output temporary_stats adalah array dua dimensi**, stack pertama berisi nanmin setiap kolom, diikuti nanmean, dan nanmax

In [9]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

## Splitting the Dataset

### Splitting the Columns

Kita harus tahu lebih dulu, mana kolom yang berisi string dan mana yang numeric. Dengan cara **mencari index kolom tersebut dengan *argwhere***

In [10]:
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
columns_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12], dtype=int64)

Kolom yang berisi string adalah kolom kedua, keempat dan seterusnya  
Sedangkan yang berisi numerik adalah kolom pertama, ketiga dan seterusnya

In [11]:
columns_numerics = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
columns_numerics

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

### Re-importing the Dataset

Kemudian, kita re-import datasetnya untuk split berdasarkan string dan numerical. Disini kita **menggunakan usecols berdasarkan indexing yang sudah dibuat sebelumnya**

In [12]:
loan_data_strings = np.genfromtxt("Loan-Data.csv", delimiter = ";", 
                         skip_header = 1, autostrip = True,
                         usecols = columns_strings,  # Data yang digunakan hanya pada kolom yang berisi string saja
                         dtype = str)  # Ubah dtypenya menjadi string untuk memastikan
loan_data_strings

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [13]:
loan_data_numerics = np.genfromtxt("Loan-Data.csv", delimiter = ";", 
                         skip_header = 1, autostrip = True,
                         usecols = columns_numerics, # Data yang digunakan hanya pada kolom yang berisi numeric saja
                         filling_values = temporary_fill)  # Untuk filler NaN, menggunakan temp_fill yang sudah dibuat diatas
loan_data_numerics

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

### The Names of the Columns

Karena sebelumnya skip_header, sekarang **kita harus menambahkan kembali headernya**. Karena tidak mungkin file yang mau dianalis atau diproses tidak memiliki header.  

In [14]:
header_full = np.genfromtxt("Loan-Data.csv", delimiter = ";", 
                         skip_footer = raw_data.shape[0],  # Kita hanya menggunakan baris pertama saja, sedangkan isi datanya tidak 
                         autostrip = True,
                         dtype = str)  
header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

Untuk memudahkan, **masing-masing tipe data dibuat variabel header sendiri** berdasarkan indexing sebelumnya

In [15]:
header_strings, header_numeric = header_full[columns_strings], header_full[columns_numerics]

In [16]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [17]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

## Creating Checkpoints:

Ini tidak ada di tutorial NumPy, namun Checkpoint adalah teknik penting saat pre-processing   
**Fungsinya untuk save progress sementara**, sebagai penjagaan jika kita tidak sengaja override nama variabel dan kesalahan lainnya  


In [18]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

Kita membuat **variabel dengan nama "checkpoint"** yang memiliki tiga parameter, file_name, header, dan data  
Lalu menggunakan bantuan **npz dan npy** untuk storingnya  
Kemudian variabel **checkpoint_variable yang berfungsi untuk load npz filenya**

In [19]:
checkpoint_test = checkpoint("test", header_strings, loan_data_strings)

Kita tes dengan memasukkan header dan data dari kolom string

In [20]:
checkpoint_test["header"]

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [21]:
checkpoint_test["data"]

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

# **Manipulating** String Columns

Langkah utama dalam pre-processing adalah membersihkan setiap kolom yang ada di data  
Penjelasan masing-masing atribut kolom ada di file ***Loan Dataset Dictionary.xlsx***  

## Issue Date

Dimulai dari kolom pertama **issue_date**

In [22]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

Dari array tersebut nama kolom *issue_d* masih terdapat typo  
Mari kita ubah menjadi **issue_date**

In [23]:
header_strings[0] = "issue_date"

Lalu kita analisis isi dari kolom *issue_date* dengan **np.unique**

In [24]:
np.unique(loan_data_strings[:,0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

Bisa dilihat, pola dari datanya sama semua. Yaitu nama bulan yang ada di tahun 2015.  
Karenanya **kita bisa extract nama bulannya saja** tanpa tahunnya dengan **np.chararray.strip**

In [26]:
loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0], "-15")

In [27]:
np.unique(loan_data_strings[:,0])

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')

Biasanya dalam analisis, bulan akan didefinisikan sebagai *integer* agar tidak memakan memori  
Buat variabel **months** yang berisi unique value dari kolom *issue_date* dengan urutan sebenarnya

In [28]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

Assign value *issue_date* sebelumnya menjadi urutan *numerical value* dari array *months* dengan **for loop**  
Ubah Januari menjadi 1, Februari menjadi 2, dst. Sedangkan NaN menjadi 0

In [29]:
for i in range(13):
        loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                          i,
                                          loan_data_strings[:,0])

In [30]:
np.unique(loan_data_strings[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

Dataset di *issue_date* sudah dalam numerical value namun masih dalam format string  
**Kita akan convert semuanya ke string di akhir pre-processing**

## Loan Status

### Term

### Grade and Subgrade

#### Filling Sub Grade

In [ ]:
for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
                                      i + '5',
                                      loan_data_strings[:,4])

#### Removing Grade

#### Converting Sub Grade

In [ ]:
keys = list(np.unique(loan_data_strings[:,3]))                         
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1)) 
dict_sub_grade = dict(zip(keys, values))

In [ ]:
for i in np.unique(loan_data_strings[:,3]):
        loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i, 
                                          dict_sub_grade[i],
                                          loan_data_strings[:,3])

### Verification Status

### URL

### State Address

In [ ]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

## Converting to Numbers

### Checkpoint 1: Strings

## Manipulating Numeric Columns

### Substitute "Filler" Values

#### ID

#### Temporary Stats

#### Funded Amount

In [ ]:
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill, 
                                  temporary_stats[0, columns_numeric[2]],
                                  loan_data_numeric[:,2])
loan_data_numeric[:,2]

#### Loaned Amount, Interest Rate, Total Payment, Installment

### Currency Change

#### The Exchange Rate

In [ ]:

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    


#### From USD to EUR

In [ ]:
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

#### Expanding the header

In [ ]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

In [ ]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

### Interest Rate

### Checkpoint 2: Numeric

In [ ]:
checkpoint_numeric['header'], checkpoint_numeric['data']

## Creating the "Complete" Dataset

## Sorting the New Dataset

## Storing the New Dataset